In [2]:
import pandas as pd
import numpy as np
import xgboost
import sklearn
from datetime import datetime, timedelta

# Import

In [4]:
df_input

,District,Latitude,Longitude,"Season(SA = Summer Autumn, WS = Winter Spring)","Rice Crop Intensity(D=Double, T=Triple)",Date of Harvest,Field size (ha),Rice Yield (kg/ha)
0,Chau_Phu,10.510542,105.248554,SA,T,2022-07-15,3.40,5500
1,Chau_Phu,10.509150,105.265098,SA,T,2022-07-15,2.43,6000
2,Chau_Phu,10.467721,105.192464,SA,D,2022-07-15,1.95,6400
3,Chau_Phu,10.494453,105.241281,SA,T,2022-07-15,4.30,6000
4,Chau_Phu,10.535058,105.252744,SA,D,2022-07-14,3.30,6400
...,...,...,...,...,...,...,...,...
552,Thoai_Son,10.364419,105.164984,WS,T,2022-04-12,7.80,6640
553,Thoai_Son,10.358094,105.189541,WS,T,2022-04-12,2.00,7200
554,Thoai_Son,10.368014,105.238516,WS,T,2022-04-12,6.20,7200
555,Thoai_Son,10.275419,105.234563,WS,T,2022-04-20,3.00,6400


In [3]:
dateparse = lambda x: datetime.strptime(x, '%d-%m-%Y')
df_input = pd.read_csv("Crop_Yield_Data_challenge_2.csv", parse_dates=['Date of Harvest'], date_parser=dateparse)
df_input.head()

,District,Latitude,Longitude,"Season(SA = Summer Autumn, WS = Winter Spring)","Rice Crop Intensity(D=Double, T=Triple)",Date of Harvest,Field size (ha),Rice Yield (kg/ha)
0,Chau_Phu,10.510542,105.248554,SA,T,2022-07-15,3.40,5500
1,Chau_Phu,10.509150,105.265098,SA,T,2022-07-15,2.43,6000
2,Chau_Phu,10.467721,105.192464,SA,D,2022-07-15,1.95,6400
3,Chau_Phu,10.494453,105.241281,SA,T,2022-07-15,4.30,6000
4,Chau_Phu,10.535058,105.252744,SA,D,2022-07-14,3.30,6400


In [20]:
df_output = pd.read_csv("Challenge_2_submission_template.csv")
df_output.head()

,ID No,District,Latitude,Longitude,"Season(SA = Summer Autumn, WS = Winter Spring)","Rice Crop Intensity(D=Double, T=Triple)",Date of Harvest,Field size (ha),Predicted Rice Yield (kg/ha)
0,1,Chau_Phu,10.542192,105.187920,WS,T,10-04-2022,1.40,NaN
1,2,Chau_Thanh,10.400189,105.331053,SA,T,15-07-2022,1.32,NaN
2,3,Chau_Phu,10.505489,105.203926,SA,D,14-07-2022,1.40,NaN
3,4,Chau_Phu,10.523520,105.138274,WS,D,10-04-2022,1.80,NaN
4,5,Thoai_Son,10.294660,105.248528,SA,T,20-07-2022,2.20,NaN


# Analyse

In [6]:
df_input.groupby("Season(SA = Summer Autumn, WS = Winter Spring)")["Rice Yield (kg/ha)"].mean()

Season(SA = Summer Autumn, WS = Winter Spring)
SA    5926.692607
WS    7234.933333
Name: Rice Yield (kg/ha), dtype: float64

In [7]:
df_input.groupby("Season(SA = Summer Autumn, WS = Winter Spring)")["Rice Yield (kg/ha)"].median()

Season(SA = Summer Autumn, WS = Winter Spring)
SA    6000.0
WS    7200.0
Name: Rice Yield (kg/ha), dtype: float64

In [13]:
df_input["Guess"] = df_input.apply(lambda x: 6000 if x["Season(SA = Summer Autumn, WS = Winter Spring)"]=="SA" else 7200, axis=1)
df_input

,District,Latitude,Longitude,"Season(SA = Summer Autumn, WS = Winter Spring)","Rice Crop Intensity(D=Double, T=Triple)",Date of Harvest,Field size (ha),Rice Yield (kg/ha),Guess
0,Chau_Phu,10.510542,105.248554,SA,T,2022-07-15,3.40,5500,6000
1,Chau_Phu,10.509150,105.265098,SA,T,2022-07-15,2.43,6000,6000
2,Chau_Phu,10.467721,105.192464,SA,D,2022-07-15,1.95,6400,6000
3,Chau_Phu,10.494453,105.241281,SA,T,2022-07-15,4.30,6000,6000
4,Chau_Phu,10.535058,105.252744,SA,D,2022-07-14,3.30,6400,6000
...,...,...,...,...,...,...,...,...,...
552,Thoai_Son,10.364419,105.164984,WS,T,2022-04-12,7.80,6640,7200
553,Thoai_Son,10.358094,105.189541,WS,T,2022-04-12,2.00,7200,7200
554,Thoai_Son,10.368014,105.238516,WS,T,2022-04-12,6.20,7200,7200
555,Thoai_Son,10.275419,105.234563,WS,T,2022-04-20,3.00,6400,7200


In [64]:
sklearn.metrics.explained_variance_score(
    np.array(df_input["Rice Yield (kg/ha)"]),
    np.array(df_input["Guess"])
)

0.6686506580309319

In [16]:
df_input["Guess_mean"] = df_input.apply(lambda x: 5926.692607 if x["Season(SA = Summer Autumn, WS = Winter Spring)"]=="SA" else 7234.933333, axis=1)
df_input

,District,Latitude,Longitude,"Season(SA = Summer Autumn, WS = Winter Spring)","Rice Crop Intensity(D=Double, T=Triple)",Date of Harvest,Field size (ha),Rice Yield (kg/ha),Guess,Guess_mean
0,Chau_Phu,10.510542,105.248554,SA,T,2022-07-15,3.40,5500,6000,5926.692607
1,Chau_Phu,10.509150,105.265098,SA,T,2022-07-15,2.43,6000,6000,5926.692607
2,Chau_Phu,10.467721,105.192464,SA,D,2022-07-15,1.95,6400,6000,5926.692607
3,Chau_Phu,10.494453,105.241281,SA,T,2022-07-15,4.30,6000,6000,5926.692607
4,Chau_Phu,10.535058,105.252744,SA,D,2022-07-14,3.30,6400,6000,5926.692607
...,...,...,...,...,...,...,...,...,...,...
552,Thoai_Son,10.364419,105.164984,WS,T,2022-04-12,7.80,6640,7200,7234.933333
553,Thoai_Son,10.358094,105.189541,WS,T,2022-04-12,2.00,7200,7200,7234.933333
554,Thoai_Son,10.368014,105.238516,WS,T,2022-04-12,6.20,7200,7200,7234.933333
555,Thoai_Son,10.275419,105.234563,WS,T,2022-04-20,3.00,6400,7200,7234.933333


In [66]:
sklearn.metrics.explained_variance_score(
    np.array(df_input["Rice Yield (kg/ha)"]),
    np.array(df_input["Guess_mean"])
)

0.6732594651571684

In [62]:
df_input[df_input["Season(SA = Summer Autumn, WS = Winter Spring)"]=="WS"]

,District,Latitude,Longitude,"Season(SA = Summer Autumn, WS = Winter Spring)","Rice Crop Intensity(D=Double, T=Triple)",Date of Harvest,Field size (ha),Rice Yield (kg/ha),Guess,Guess_mean,Guess mean mea
73,Chau_Phu,10.510542,105.248554,WS,T,2022-04-01,3.40,6800,7200,7234.933333,7261.693548
74,Chau_Phu,10.509150,105.265098,WS,T,2022-04-01,2.43,7200,7200,7234.933333,7261.693548
75,Chau_Phu,10.501829,105.273585,WS,T,2022-04-01,3.50,8000,7200,7234.933333,7261.693548
76,Chau_Phu,10.485464,105.251956,WS,T,2022-04-02,2.00,7040,7200,7234.933333,7261.693548
77,Chau_Phu,10.494453,105.241281,WS,T,2022-04-01,4.30,7600,7200,7234.933333,7261.693548
...,...,...,...,...,...,...,...,...,...,...,...
552,Thoai_Son,10.364419,105.164984,WS,T,2022-04-12,7.80,6640,7200,7234.933333,7261.693548
553,Thoai_Son,10.358094,105.189541,WS,T,2022-04-12,2.00,7200,7200,7234.933333,7261.693548
554,Thoai_Son,10.368014,105.238516,WS,T,2022-04-12,6.20,7200,7200,7234.933333,7261.693548
555,Thoai_Son,10.275419,105.234563,WS,T,2022-04-20,3.00,6400,7200,7234.933333,7261.693548


In [57]:
df_input[df_input["Season(SA = Summer Autumn, WS = Winter Spring)"]=="WS"].groupby("Rice Crop Intensity(D=Double, T=Triple)").median()

/tmp/ipykernel_33335/1543193907.py:1: FutureWarning: The default value of numeric_only in DataFrameGroupBy.median is deprecated. In a future version, numeric_only will default to False. Either specify numeric_only or select only columns which should be valid for the function.
  df_input[df_input["Season(SA = Summer Autumn, WS = Winter Spring)"]=="WS"].groupby("Rice Crop Intensity(D=Double, T=Triple)").median()


,Latitude,Longitude,Field size (ha),Rice Yield (kg/ha),Guess,Guess_mean,Guess mean mea
"Rice Crop Intensity(D=Double, T=Triple)",,,,,,,
D,10.473541,105.173829,2.305,7200.0,7200.0,7234.933333,7107.307692
T,10.403793,105.210650,2.200,7200.0,7200.0,7234.933333,7261.693548


In [42]:
def conditions(x):
    if x["Season(SA = Summer Autumn, WS = Winter Spring)"]=="SA":
        if x["Rice Crop Intensity(D=Double, T=Triple)"] == "D":
            return 6066.857143
        elif x["Rice Crop Intensity(D=Double, T=Triple)"] == "T":
            return 5904.594595
    elif x["Season(SA = Summer Autumn, WS = Winter Spring)"]=="WS":
        if x["Rice Crop Intensity(D=Double, T=Triple)"] == "D":
            return 7107.307692
        elif x["Rice Crop Intensity(D=Double, T=Triple)"] == "T":
            return 7261.693548
    print(x)
    return 0

In [52]:
df_input["Guess mean mea"] = df_input.apply(conditions, axis=1)
df_input.head()

,District,Latitude,Longitude,"Season(SA = Summer Autumn, WS = Winter Spring)","Rice Crop Intensity(D=Double, T=Triple)",Date of Harvest,Field size (ha),Rice Yield (kg/ha),Guess,Guess_mean,Guess mean mea
0,Chau_Phu,10.510542,105.248554,SA,T,2022-07-15,3.40,5500,6000,5926.692607,5904.594595
1,Chau_Phu,10.509150,105.265098,SA,T,2022-07-15,2.43,6000,6000,5926.692607,5904.594595
2,Chau_Phu,10.467721,105.192464,SA,D,2022-07-15,1.95,6400,6000,5926.692607,6066.857143
3,Chau_Phu,10.494453,105.241281,SA,T,2022-07-15,4.30,6000,6000,5926.692607,5904.594595
4,Chau_Phu,10.535058,105.252744,SA,D,2022-07-14,3.30,6400,6000,5926.692607,6066.857143


In [67]:
sklearn.metrics.explained_variance_score(
    np.array(df_input["Rice Yield (kg/ha)"]),
    np.array(df_input["Guess mean mea"])
)

0.6784334404847655

# Prediction

In [55]:
df_output["Predicted Rice Yield (kg/ha)"] = df_output.apply(conditions, axis=1)
df_output

,ID No,District,Latitude,Longitude,"Season(SA = Summer Autumn, WS = Winter Spring)","Rice Crop Intensity(D=Double, T=Triple)",Date of Harvest,Field size (ha),Predicted Rice Yield (kg/ha)
0,1,Chau_Phu,10.542192,105.187920,WS,T,10-04-2022,1.40,7261.693548
1,2,Chau_Thanh,10.400189,105.331053,SA,T,15-07-2022,1.32,5904.594595
2,3,Chau_Phu,10.505489,105.203926,SA,D,14-07-2022,1.40,6066.857143
3,4,Chau_Phu,10.523520,105.138274,WS,D,10-04-2022,1.80,7107.307692
4,5,Thoai_Son,10.294660,105.248528,SA,T,20-07-2022,2.20,5904.594595
...,...,...,...,...,...,...,...,...,...
95,96,Chau_Thanh,10.435839,105.132981,SA,D,26-07-2022,1.21,6066.857143
96,97,Chau_Phu,10.529357,105.147388,WS,T,10-04-2022,2.00,7261.693548
97,98,Chau_Thanh,10.452537,105.205118,SA,T,20-07-2022,5.50,5904.594595
98,99,Chau_Thanh,10.394341,105.126836,SA,T,14-07-2022,4.40,5904.594595


In [56]:
df_output.to_csv("2-mean mean.csv", index=False)